# Pipeline Testing Demonstration

This notebook demonstrates how to test individual components of the GridSource pipeline using our pytest testing framework.

## Learning Objectives
- Understand how to test pipeline components independently
- Learn to use mock data for testing without external dependencies
- Validate data transformations and ML model training
- Debug pipeline issues using isolated testing

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json

# Import GridSource package components
# Note: Install package first with: pip install -e .
from gridsource.tests.unit.extraction_functions import (
    extract_eia_electricity_data_test,
    extract_weather_data_test,
    extract_fred_data_test,
    create_ml_features_test,
    transform_eia_data,
    transform_weather_data,
    transform_fred_data
)

# Set up plotting
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

print("✅ Testing framework imported successfully")

## 1. Testing Data Extraction Functions

Let's test each data extraction function independently using mock data.

In [ ]:
# Test EIA data transformation with sample data
sample_eia_response = {
    'response': {
        'data': [
            {
                'period': '2025-05-23',
                'fueltype': 'NG',
                'value': '25000.5',
                'respondent': 'CAL'
            },
            {
                'period': '2025-05-23',
                'fueltype': 'SUN',
                'value': '15000.0',
                'respondent': 'CAL'
            },
            {
                'period': '2025-05-22',
                'fueltype': 'NG',
                'value': '26000.0',
                'respondent': 'CAL'
            }
        ]
    }
}

print("🧪 Testing EIA data transformation...")
eia_df = transform_eia_data(sample_eia_response)

print(f"✅ EIA transformation successful:")
print(f"  Shape: {eia_df.shape}")
print(f"  Columns: {list(eia_df.columns)}")
print(f"  Data types: {dict(eia_df.dtypes)}")
print("\nFirst 3 rows:")
print(eia_df.head(3))

In [ ]:
# Test weather data transformation
sample_weather_response = {
    'properties': {
        'periods': [
            {
                'name': 'Today',
                'startTime': '2025-05-23T06:00:00-07:00',
                'temperature': 72,
                'windSpeed': '10 mph',
                'shortForecast': 'Partly Cloudy'
            },
            {
                'name': 'Tonight',
                'startTime': '2025-05-23T18:00:00-07:00',
                'temperature': 58,
                'windSpeed': '5 mph',
                'shortForecast': 'Clear'
            }
        ]
    }
}

print("🧪 Testing weather data transformation...")
weather_df = transform_weather_data(sample_weather_response)

print(f"✅ Weather transformation successful:")
print(f"  Shape: {weather_df.shape}")
print(f"  Columns: {list(weather_df.columns)}")
print("\nFirst 2 rows:")
print(weather_df)

In [ ]:
# Test FRED data transformation
sample_fred_response = {
    'observations': [
        {'date': '2025-05-01', 'value': '102.5'},
        {'date': '2025-04-01', 'value': '101.8'},
        {'date': '2025-03-01', 'value': '101.2'},
        {'date': '2025-02-01', 'value': '.'} # Missing value test
    ]
}

print("🧪 Testing FRED data transformation...")
fred_df = transform_fred_data(sample_fred_response, 'industrial_production_index')

print(f"✅ FRED transformation successful:")
print(f"  Shape: {fred_df.shape}")
print(f"  Columns: {list(fred_df.columns)}")
print("\nAll rows (note missing value excluded):")
print(fred_df)

## 2. Testing ML Feature Creation

Now let's test the feature engineering process that combines all data sources.

In [ ]:
# Create sample datasets for feature engineering test
print("🔧 Creating sample datasets for feature engineering...")

# Sample EIA data (multiple fuel types)
test_eia_data = pd.DataFrame({
    'date': ['2025-05-23', '2025-05-23', '2025-05-22', '2025-05-22'],
    'fuel_type': ['NG', 'SUN', 'NG', 'SUN'],
    'generation_mwh': [25000, 15000, 26000, 14000],
    'data_source': ['EIA'] * 4
})
test_eia_data['date'] = pd.to_datetime(test_eia_data['date'])

# Sample weather data
test_weather_data = pd.DataFrame({
    'date': ['2025-05-23', '2025-05-22'],
    'temperature_f': [72, 68],
    'wind_speed': [10, 8],
    'forecast': ['Partly Cloudy', 'Clear'],
    'data_source': ['NOAA'] * 2
})
test_weather_data['date'] = pd.to_datetime(test_weather_data['date'])

# Sample economic data
test_economic_data = pd.DataFrame({
    'date': ['2025-05-23', '2025-05-22'],
    'indicator': ['crude_oil_price_wti', 'crude_oil_price_wti'],
    'value': [70.5, 71.0],
    'data_source': ['FRED'] * 2
})
test_economic_data['date'] = pd.to_datetime(test_economic_data['date'])

# Sample price data
test_price_data = pd.DataFrame({
    'date': ['2025-05-23', '2025-05-22'],
    'price_per_mwh': [45.0, 47.0],
    'data_source': ['SIMULATED'] * 2
})
test_price_data['date'] = pd.to_datetime(test_price_data['date'])

print("✅ Sample datasets created")
print(f"  EIA: {len(test_eia_data)} records")
print(f"  Weather: {len(test_weather_data)} records")
print(f"  Economic: {len(test_economic_data)} records")
print(f"  Price: {len(test_price_data)} records")

In [ ]:
# Test feature engineering
print("🧪 Testing ML feature creation...")

ml_features = create_ml_features_test(
    test_eia_data, 
    test_weather_data, 
    test_economic_data, 
    test_price_data
)

print(f"✅ ML feature creation successful:")
print(f"  Shape: {ml_features.shape}")
print(f"  Columns: {list(ml_features.columns)}")
print("\nFeature data:")
print(ml_features)

In [ ]:
# Validate feature engineering results
print("🔍 Validating feature engineering results...")

# Check data types
print("\nData Types:")
for col, dtype in ml_features.dtypes.items():
    print(f"  {col}: {dtype}")

# Check for missing values
print("\nMissing Values:")
missing = ml_features.isnull().sum()
for col, count in missing.items():
    if count > 0:
        print(f"  {col}: {count} missing")
    else:
        print(f"  {col}: ✅ No missing values")

# Check value ranges
print("\nValue Ranges:")
numeric_cols = ml_features.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col != 'date':
        min_val, max_val = ml_features[col].min(), ml_features[col].max()
        print(f"  {col}: {min_val:.2f} to {max_val:.2f}")

## 3. Testing ML Model Training

Let's test the machine learning model training component.

In [ ]:
# Import ML training class
from sagemaker.train import LiquidityForecastingModel

print("🧪 Testing ML model initialization...")

# Test different model types
model_types = ['linear_regression', 'ridge', 'random_forest']

for model_type in model_types:
    try:
        model = LiquidityForecastingModel(model_type=model_type, random_state=42)
        print(f"  ✅ {model_type}: Initialized successfully")
    except Exception as e:
        print(f"  ❌ {model_type}: Failed - {str(e)}")

print("\n✅ Model initialization tests complete")

In [ ]:
# Create larger sample dataset for ML training
print("🔧 Creating sample training dataset...")

# Generate 30 days of synthetic training data
dates = pd.date_range(start='2025-04-24', end='2025-05-23', freq='D')
np.random.seed(42)  # For reproducible results

training_data = []
for i, date in enumerate(dates):
    # Generate realistic but synthetic features
    base_generation = 75000 + np.random.normal(0, 5000)
    temperature = 65 + 15 * np.sin(2 * np.pi * i / 30) + np.random.normal(0, 3)
    oil_price = 70 + 5 * np.sin(2 * np.pi * i / 60) + np.random.normal(0, 2)
    industrial_index = 101.5 + 0.1 * i + np.random.normal(0, 0.5)
    electricity_price = 45 + 5 * np.sin(2 * np.pi * i / 20) + np.random.normal(0, 1)
    
    # Target variable (liquidity need) - simplified relationship
    liquidity_need = (
        base_generation * 0.002 +  # Generation factor
        oil_price * 1.5 +          # Oil price factor
        temperature * 0.5 +        # Temperature factor
        np.random.normal(0, 10)    # Random variation
    )
    
    training_data.append({
        'total_generation_mwh': base_generation,
        'avg_temperature_f': temperature,
        'oil_price_usd': oil_price,
        'industrial_production_index': industrial_index,
        'avg_electricity_price': electricity_price,
        'liquidity_need_millions': liquidity_need
    })

training_df = pd.DataFrame(training_data)

print(f"✅ Training dataset created: {len(training_df)} samples")
print(f"Features: {list(training_df.columns)}")
print("\nDataset statistics:")
print(training_df.describe().round(2))

In [ ]:
# Test model training
print("🧪 Testing model training process...")

# Initialize model
model = LiquidityForecastingModel(model_type='linear_regression', random_state=42)

# Prepare features and target
feature_columns = [
    'total_generation_mwh',
    'avg_temperature_f',
    'oil_price_usd',
    'industrial_production_index',
    'avg_electricity_price'
]

X = training_df[feature_columns]
y = training_df['liquidity_need_millions']

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")

# Set feature names and train
model.feature_names = feature_columns
model.train(X, y, test_size=0.3)

print("\n✅ Model training completed successfully!")

# Display training metrics
print("\n📊 Training Metrics:")
train_metrics = model.training_metrics['train']
val_metrics = model.training_metrics['validation']

print(f"Training:")
print(f"  MAE: ${train_metrics['mae']:.2f}M")
print(f"  RMSE: ${train_metrics['rmse']:.2f}M")
print(f"  R²: {train_metrics['r2']:.4f}")
print(f"  MAPE: {train_metrics['mape']:.2f}%")

print(f"\nValidation:")
print(f"  MAE: ${val_metrics['mae']:.2f}M")
print(f"  RMSE: ${val_metrics['rmse']:.2f}M")
print(f"  R²: {val_metrics['r2']:.4f}")
print(f"  MAPE: {val_metrics['mape']:.2f}%")

In [ ]:
# Test model predictions
print("🧪 Testing model predictions...")

# Make predictions on first 5 samples
test_features = X.head(5)
predictions = model.predict(test_features)
actual_values = y.head(5).values

print("Prediction vs Actual Comparison:")
for i in range(5):
    pred = predictions[i]
    actual = actual_values[i]
    error = abs(pred - actual)
    print(f"  Sample {i+1}: Predicted ${pred:.2f}M, Actual ${actual:.2f}M (Error: ${error:.2f}M)")

# Calculate prediction accuracy
avg_error = np.mean(np.abs(predictions - actual_values))
print(f"\nAverage prediction error: ${avg_error:.2f}M")

print("\n✅ Model prediction tests complete")

## 4. Visualizing Test Results

Let's create some visualizations to better understand our test results.

In [ ]:
# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('ML Model Testing Results', fontsize=16)

# 1. Training data distribution
training_df['liquidity_need_millions'].hist(bins=15, ax=axes[0,0], alpha=0.7, color='skyblue')
axes[0,0].set_title('Target Variable Distribution')
axes[0,0].set_xlabel('Liquidity Need (Millions USD)')
axes[0,0].set_ylabel('Frequency')

# 2. Feature correlations with target
correlations = []
for feature in feature_columns:
    corr = training_df[feature].corr(training_df['liquidity_need_millions'])
    correlations.append(corr)

axes[0,1].barh(feature_columns, correlations, color='lightcoral')
axes[0,1].set_title('Feature Correlations with Target')
axes[0,1].set_xlabel('Correlation Coefficient')

# 3. Prediction vs Actual scatter plot
all_predictions = model.predict(X)
axes[1,0].scatter(y, all_predictions, alpha=0.6)
axes[1,0].plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
axes[1,0].set_xlabel('Actual Liquidity Need (Millions USD)')
axes[1,0].set_ylabel('Predicted Liquidity Need (Millions USD)')
axes[1,0].set_title('Predictions vs Actual Values')

# 4. Prediction errors
errors = all_predictions - y
axes[1,1].hist(errors, bins=15, alpha=0.7, color='lightgreen')
axes[1,1].set_title('Prediction Error Distribution')
axes[1,1].set_xlabel('Error (Predicted - Actual)')
axes[1,1].set_ylabel('Frequency')
axes[1,1].axvline(x=0, color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 5. Running Pytest Tests

Let's demonstrate how to run the actual pytest tests from within the notebook.

In [ ]:
# Run specific unit tests
import subprocess
import sys

print("🧪 Running unit tests for data extraction...")

# Run a specific test function
test_command = [
    sys.executable, "-m", "pytest", 
    "../tests/unit/test_data_extraction.py::TestDataExtraction::test_eia_data_transformation",
    "-v"
]

try:
    result = subprocess.run(test_command, 
                          capture_output=True, 
                          text=True, 
                          cwd="..")
    
    print("Test Output:")
    print(result.stdout)
    
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
    print(f"\nTest exit code: {result.returncode}")
    print("✅ Unit test execution complete" if result.returncode == 0 else "❌ Unit test failed")
    
except Exception as e:
    print(f"❌ Error running tests: {str(e)}")
    print("💡 Make sure pytest is installed: pip install -r ../tests/requirements.txt")

## 6. Test Coverage Analysis

Let's analyze what parts of our pipeline are covered by tests.

In [ ]:
# Analyze test coverage
print("📊 Test Coverage Analysis")
print("=" * 30)

# Define pipeline components and their test status
pipeline_components = {
    'Data Extraction': {
        'EIA API': '✅ Tested',
        'Weather API': '✅ Tested', 
        'FRED API': '✅ Tested',
        'Energy Prices': '✅ Tested'
    },
    'Data Transformation': {
        'EIA Transform': '✅ Tested',
        'Weather Transform': '✅ Tested',
        'FRED Transform': '✅ Tested',
        'Feature Engineering': '✅ Tested'
    },
    'ML Pipeline': {
        'Model Training': '✅ Tested',
        'Model Prediction': '✅ Tested',
        'Model Persistence': '✅ Tested',
        'Performance Metrics': '✅ Tested'
    },
    'Integration': {
        'End-to-End Flow': '✅ Tested',
        'Error Handling': '✅ Tested',
        'Data Validation': '✅ Tested',
        'S3 Integration': '✅ Tested'
    },
    'Not Yet Tested': {
        'Snowflake Integration': '⚠️ Manual Testing',
        'Airflow DAG': '⚠️ Manual Testing',
        'SageMaker Deployment': '⚠️ Manual Testing',
        'Power BI Views': '⚠️ Manual Testing'
    }
}

for category, components in pipeline_components.items():
    print(f"\n{category}:")
    for component, status in components.items():
        print(f"  {component}: {status}")

# Calculate coverage percentage
total_components = sum(len(components) for components in pipeline_components.values())
tested_components = sum(
    sum(1 for status in components.values() if '✅' in status)
    for components in pipeline_components.values()
)

coverage_percentage = (tested_components / total_components) * 100

print(f"\n📈 Overall Test Coverage: {coverage_percentage:.1f}% ({tested_components}/{total_components} components)")

## 7. Testing Best Practices Demonstrated

This notebook has demonstrated several testing best practices:

In [ ]:
print("🎯 Testing Best Practices Demonstrated:")
print("=" * 40)

best_practices = {
    '✅ Unit Testing': [
        'Test individual functions in isolation',
        'Use mock data to avoid external dependencies',
        'Test both success and failure scenarios',
        'Validate data types and shapes'
    ],
    '✅ Integration Testing': [
        'Test component interactions',
        'Validate end-to-end data flow',
        'Test with realistic data volumes',
        'Check error propagation'
    ],
    '✅ Data Validation': [
        'Check for missing values',
        'Validate data ranges and types',
        'Test data transformation accuracy',
        'Verify feature engineering logic'
    ],
    '✅ ML Model Testing': [
        'Test model initialization',
        'Validate training process',
        'Check prediction functionality',
        'Monitor performance metrics'
    ],
    '✅ Reproducibility': [
        'Use fixed random seeds',
        'Version control test data',
        'Document test expectations',
        'Consistent test environments'
    ]
}

for category, practices in best_practices.items():
    print(f"\n{category}:")
    for practice in practices:
        print(f"  • {practice}")

print("\n🎉 Testing framework successfully demonstrates all key practices!")

## Summary

In this notebook, we:

1. **✅ Tested data extraction functions** with mock API responses
2. **✅ Validated data transformations** for all external data sources
3. **✅ Tested ML feature engineering** with combined datasets
4. **✅ Verified ML model training** and prediction functionality
5. **✅ Demonstrated pytest integration** for automated testing
6. **✅ Analyzed test coverage** across pipeline components
7. **✅ Showcased testing best practices** for data pipelines

## Key Benefits of This Testing Approach

- **🔍 Debugging**: Easy to isolate and fix issues in specific components
- **🚀 Development Speed**: Test changes quickly without running entire pipeline
- **📊 Quality Assurance**: Catch data quality issues early
- **🔄 Reproducibility**: Consistent results across environments
- **📈 Confidence**: Deploy with confidence knowing components work

## Next Steps

1. Run full test suite: `python tests/run_tests.py --type all`
2. Add integration tests for Snowflake and Airflow components
3. Set up continuous integration (CI) for automated testing
4. Monitor test coverage as pipeline evolves
5. Add performance and load testing for production deployment